## Our Twitter Reply Bot

In class we are going to write a simple Twitter bot. We'd like you to work on the "brains" of the bot, coming up with the bot interactions: what does the bot do? what does the user need to send to the bot? how does the bot respond? what data does it need? etc.

You'll be focusing on the logic of the bot but we need some "scaffolding" here - we need the code that will listen to @mentions (for our bot) and will respond to users.

There are a few ways for us to write our Twitter bot. In our case, we want our bot to reply to anyone who sends an @mention to our bot, like: 

`@covidbot what's the covid case count in New York`.

### Using Twitter's Realtime API

To build our reply bot, we can take two approaches:

1. Use Twitter's [Realtime ("Streaming") API](https://developer.twitter.com/en/docs/tweets/filter-realtime/overview/statuses-filter) to be notified of @mentions in realtime, or
2. Use Twitter's [`statuses/mentions_timeline` API](https://developer.twitter.com/en/docs/tweets/timelines/api-reference/get-statuses-mentions_timeline) to get all @mentions for a user. For this method, you'd need to call the API every few seconds/minutes checking for new @mentions.

There are pro's and con's to both approach here, but for fun, let's use the Realtime/Streaming APIs. **Note** the example in this notebook is a simple example and would need to be modified to work well as a bot that runs 24x7 without falling over! There is no error checking and I've certainly missed some corner cases - we can discuss in class which of the two approaches you'd take to make this a real bot!


With Twitter's Streaming api, you can "listen to" or "track" multiple terms or phrases and twitter will send those tweets to us in realtime! it's up to us to respond to the tweets as we see fit. We can also save the tweets in a file, database...or even ignore them if you want.

So, you could "track" phrases like `covid` or `coronavirus` if you want and you'll get a sampling of realtime tweets that contain those phrases. You can also ask to track mentions of `@covidbot` or whatever your bot name is (which is what we'll do). More information about filtering realtime tweets can be found here: https://developer.twitter.com/en/docs/tweets/filter-realtime/overview/statuses-filter


### Our Bot Function

As Mark mentioned in our class notebook, we' want you to write a function that takes a tweet's text and a screen name (or a user summoning our bot) and returns the text that we'd like to send back to the user. 

For example....if we're building a simple "echo" bot that just takes the the incoming tweet text and replies back to the user saying `hi` and echo'ing back what they tweeted at us, we could do this:

In [ ]:
def bot(tweet_text, screen_name):
    ''' this function takes the screen name and text of an incoming tweet
        and simply returns the text that we want to reply with
    '''

    # just reply with what was tweeted at us
    # the format of our reply will be:
    # "hi @person: [the text of their tweet will go here]"
    reply = 'hi @' + screen_name + ': ' + tweet_text

    return reply

We'll use Tweepy to use the Twitter streaming API. You've already installed it but it doesn't hurt to do it again...

In [ ]:
!pip install tweepy

This is the scaffolding of the reply bot, using Tweepy's `StreamListener` to listen for people @mentioning our bot and then replying to them. There are some new concepts in here that we can cover in class or over the coming week...but have a read of the code:

In [ ]:
''' reply bot
'''

import tweepy

class BotStreamListener(tweepy.StreamListener):
    ''' this is our simple twitter bot "listener"
        by "listener", we mean that it will listen to twitter for certain tweets
        and we can respond/reply when we receive tweets that we are interest in
    '''
    def __init__(self, api):
        self.api = api

        # make an API call to twitter to get information about the
        # authenticated user that is running this listener
        # we'll use this infor below
        self.me = api.me()

    def on_status(self, tweet):
        ''' this is called by tweepy's StreamListener whenever we get a tweet that matches our filter rules
            tweet is a Tweepy tweet object
        '''

        # let's print out the tweet that just came in
        print('----'*20)
        print('incoming tweet:')
        print(tweet.user)
        print(tweet.text)
        print('----'*20)

        # ignore any tweets sent by/from us
        # without this, we may endless reply to ourself :-)
        if tweet.user.id_str == self.me.id_str:
            print('ignoring tweets from our bot')
            return

        # call the bot function that we've defined above
        # this wil pass in the tweet text and screen name of the tweet(er)
        # and whatever text the bot() function returns will be sent
        # back to the screen_name as a twitter reply
        reply = bot(tweet.text, tweet.user.screen_name)

        if reply:
            # reply to the user
            print(f'replying with: {reply}')

            # reply to the user with the "reply" text
            # and, by specifying the incoming tweet "id string",
            # we are telling tweepy, and the twitter api, that this is a reply
            api.update_status(reply, tweet.id_str)
        else:
            # hmmm...nothing to reply with so let's not even try
            print('nothing to reply with! we wont send a reply')


In [ ]:
# auth keys and secrets for our bot
# you can use your own bot keys here if you'd like to run this
consumer_key = 'Xqw8vDxc7PhOrvQMiOILjkmgS'
consumer_secret = 'oqFvZo1G3jnnSVSIztOdNj1bmAnCC2roDv9CnfF0sOhF8G0aHt'
access_token = '961659080266993665-cxji3bUl4B1bs01ZejiCpyEKwgIs2cl'
access_token_secret = 'IGPQccmYAWRNOW040BR6MEzgFylcO3qO5gpvpM6i97jzw'

# set up the auth object like we've always done with tweetpy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [ ]:
# create out bot "listener" object
bot_listener = BotStreamListener(api)

# a bot object by telling tweepy that we'd like to use twitter "streaming" api
my_bot = tweepy.Stream(auth=api.auth, listener=bot_listener)

# phrase(s) to filter for using the 'track' rules:
# https://developer.twitter.com/en/docs/tweets/filter-realtime/guides/basic-stream-parameters
# call twitter to get info about me (this account)
me = api.me()
# now, create the string which we will filter for below: "@stangellbaugh" (or whatever the auth'd user's screen_name is)
#my_screen_name = f'@{me.screen_name}'
my_screen_name = '@' + me.screen_name

# start our bot listener and listen for @mentions for the auth'd user (key/secrets above)
my_bot.filter(track=[str(my_screen_name)])

When you run this code above, it will run "forever" until you stop it, or it raises an Exception and crashes :-)

**Make sure you stop and restart this cell if you make any edits to the `bot()` function or the listener**